In [1]:
import visa
import serial
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rm=visa.ResourceManager()
device=[]
device=rm.list_resources()

In [3]:
class Instrumento:

    def __init__(self, serialno):
        self.serial = serialno
        self.instr = rm.open_resource(serialno)


class WaveGenerator(Instrumento):

    ##@Feat()
    def who_am_i(self):
        idn = self.instr.query('*IDN?')
        return idn.strip()

    def which_freq_mod(self):
        mod = self.instr.query('SOURce1:FREQuency:MODE?')
        return mod.strip()

    # def which_volt_amp(self):
    #     dum = self.instr.query('SOURce1:VOLTage:LEVel:IMMediate:AMPLitude?')
    #     return float(dum)
    #
    # def which_volt_min(self):
    #     dum = self.instr.query('SOURce1:VOLTage:LEVel:IMMediate:AMPLitude? MINimum')
    #     return float(dum)
    #
    # def which_volt_max(self):
    #     dum = self.instr.query('SOURce1:VOLTage:LEVel:IMMediate:AMPLitude? MAXimum')
    #     return float(dum)
    #
    # def which_volt_unit(self):
    #     return self.instr.query('SOURce1:VOLTage:UNIT?')

    def which_volt_amp(self):
        amp = self.instr.query('SOURce1:VOLTage:LEVel:IMMediate:AMPLitude?')
        ampmin = self.instr.query('SOURce1:VOLTage:LEVel:IMMediate:AMPLitude? MINimum')
        ampmax = self.instr.query('SOURce1:VOLTage:LEVel:IMMediate:AMPLitude? MAXimum')
        ampunit = self.instr.query('SOURce1:VOLTage:UNIT?')
        return float(amp), float(ampmin), float(ampmax), ampunit.strip()

    def which_volt_offset(self):
        voff = self.instr.query('SOURce1:VOLTage:OFFSet?')
        voffmin = self.instr.query('SOURce1:VOLTage:OFFSet? MINimum')
        voffmax = self.instr.query('SOURce1:VOLTage:OFFSet? MAXimum')
        return float(voff), float(voffmin), float(voffmax)

    def freq_mod(self, modtype):
        # options:
        # CW|SWEep
        print('Setting new freq mode:')
        if modtype == 'CW':
            print(' ==> CW (continuous wave)')
        if modtype == 'SWEep':
            print(' ==> Sweep')
        return self.instr.write('SOURce1:FREQuency:MODE {}'.format(modtype))

    def freq_fix(self,freq):
        return self.instr.write('SOURce1:FREQuency:FIXed {}'.format(freq))

    def func_shape(self,ishape):
        # options:
        # SINusoid|SQUare|PULSe|RAMP|PRNoise|DC|SINC|GAUSsian|LORentz|ERISe|EDECay|HAVersine
        print('Setting new function: \n ==> {} function'.format(ishape))
        return self.instr.write('SOURce1:FUNCtion:SHAPe {}'.format(ishape))

    def voltaje(self,amplitude,units):
        # options:
        # VPP|VRMS|DBM
        if amplitude > voltmax:
            raise ValueError(' *** input amplitude should be smaller than {} *** '.format(voltmax))
        if amplitude < voltmin:
            raise ValueError(' *** input amplitude should be greater than {} *** '.format(voltmin))
        newvolt=str(amplitude)+" "+units
        print('Setting new voltaje: \n ==> {} {}'.format(amplitude,units))
        return self.instr.write('SOURce1:VOLTage:LEVel:IMMediate:AMPLitude {} {}'.format(amplitude,units))

    def volt_offset(self,voffset,units):
        print('Setting new offset voltaje: \n ==> {} {}'.format(voffset,units))
        return self.instr.write('SOURce1:VOLTage:LEVel:IMMediate:OFFSet {} {}'.format(voffset,units))


    def arduino(com='COM4'):
        puerto_serie=serial.Serial(com,9600,timeout=0)
        lectura=puerto_serie.readline().strip()
        puerto_serie.close() 
        return lectura

In [142]:

# define instrument
my_instrument = WaveGenerator(device[0])

whoami = my_instrument.who_am_i()
deffreqmod = my_instrument.which_freq_mod()
# whichvoltamp = my_instrument.which_volt_amp()
# whichvoltunit = my_instrument.which_volt_unit()
# voltmin = my_instrument.which_volt_min()
# voltmax = my_instrument.which_volt_max()
defvoltamp, defvoltmin, defvoltmax, defvoltunit = my_instrument.which_volt_amp()
voltmin = float(defvoltmin)
voltmax = float(defvoltmax)
defvoffset, defvoffmin, defvoffmax = my_instrument.which_volt_offset()

print('Current instrument:',whoami)
print('DEFAULT VALUES:')
print('---------------')
print(' Frequency mode:',deffreqmod)
print(' Voltaje:',defvoltamp,defvoltunit)
print('     min:',voltmin,defvoltunit)
print('     max:',voltmax,defvoltunit)
print(' Offset:',defvoffset)

print('\nNEW VALUES:')
print('-------------')
# set frequency mode:
my_instrument.freq_mod('CW')
# set function shape
my_instrument.func_shape('SIN')
# set fixed frequency value
my_instrument.freq_fix('100Hz')
# set voltaje amplitude
my_instrument.voltaje(1,'VPP')
# set voltaje offset
my_instrument.volt_offset(0,'V')

Current instrument: TEKTRONIX,AFG3021B,C034167,SCPI:99.0 FV:3.1.1
DEFAULT VALUES:
---------------
 Frequency mode: CW
 Voltaje: 1.0 VPP
     min: 0.02 VPP
     max: 19.0 VPP
 Offset: 0.5

NEW VALUES:
-------------
Setting new freq mode:
 ==> CW (continuous wave)
Setting new function: 
 ==> SIN function
Setting new voltaje: 
 ==> 1 VPP
Setting new offset voltaje: 
 ==> 0 V


(44, <StatusCode.success: 0>)

## codigo arduino:

int PinLectura = 0;
float Volt;

void setup() {
  
  Serial.begin(9600);
}

void loop() {
  
  Volt = (analogRead(PinLectura))*5.0/1023;  // Leer voltaje pin A0
  Serial.println(Volt);

  delay(100);
}

# set frequency to write and read with arduino

In [143]:
fmin=100
fmax=100
nfreq=1
fvals=np.linspace(fmin,fmax,nfreq)
npts=100

In [186]:
puerto_serie.close()

In [145]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [189]:
puerto_serie=serial.Serial('COM4',9600)
funcs=[]
wave=[]
npts=300
for j in range(npts):
    lectura=puerto_serie.readline().strip()
    flec=float(lectura)
    wave.append(flec)
funcs.append(wave)
puerto_serie.close()

In [190]:
plt.plot(wave,'ko',linestyle='solid')
plt.show()

In [191]:
np.savetxt("arduinoPaso50-PWM175.dat",wave)

In [146]:
puerto_serie=serial.Serial('COM4',9600)
funcs=[]
for i in fvals:
    newfreq=str(i)+"Hz"
    my_instrument.freq_fix(newfreq)
    wave=[]
    for j in range(npts):
        lectura=puerto_serie.readline().strip()
        flec=float(lectura)
        wave.append(flec)
    funcs.append(wave)
puerto_serie.close()

In [181]:
puerto_serie.close()

In [173]:
plt.plot(funcs,'ko',linestyle='solid')
plt.show()

In [149]:
for i in range(nfreq):
    print(i)
    np.savetxt(str(fvals[i])+"Hz_1VPP_delay100_recortada.dat",funcs[i])

0


In [85]:
fun0=np.loadtxt("1.0Hz_5VPP_delay100.dat")
fun1=np.loadtxt("1.0Hz_5VPP_delay10.dat")
fun2=np.loadtxt("1.0Hz_5VPP_delay1.dat")

In [86]:
plt.plot(fun0,'ko',linestyle='solid')
plt.plot(fun1,'ro',linestyle='solid')
plt.plot(fun2,'go',linestyle='solid')

plt.show()

In [112]:
fun0=np.loadtxt("100.0Hz_7VPP_delay100.dat")
plt.plot(fun0,'ko',linestyle='solid')
plt.show()

# set frequency to write arduino

In [151]:
import visa
from matplotlib import pyplot as plt
import numpy as np
import os, sys

In [169]:
rm=visa.ResourceManager()
device=[]
device=rm.list_resources()

#print(device)

class Osciloscopio:
    
    def __init__(self, devnumber):
        self.devnumber = devnumber
        self.oscilo = rm.open_resource(devnumber)
    
    def quien_soy(self):
        return self.oscilo.query('*IDN?')
    
    def auto_set(self):
        self.oscilo.write('AUTOSet')
    
    def trigger_cuanto_es(self):
        return self.oscilo.query('TRIGger?')
    
    def trigger_set(self, voltaje_trigger):
        self.oscilo.write('TRIGger:MAIn:LEVel'+str(voltaje_trigger))
    
    def trigger_pulso(self, channel):
        self.oscilo.write('TRIGger:MAIn:PULse:SOUrce '+str(channel)) 
    
    def tiempo_que_usas(self):
        return self.oscilo.query('HORizontal:DELay?')
    
    def tiempo_set(self, tiempo):
        self.oscilo.write('HORizontal:MAIn:SCAle '+str(tiempo))
    
    def voltaje_ch1(self, channel, voltaje):
        self.oscilo.write('CH'+str(channel)+':SCAle '+str(voltaje))
        
    def set_del_ch1(self, channel):
        return self.oscilo.query('CH'+str(channel)+'?')	
    
    def pantalla(self,nombre):
        xze, xin, yze, ymu, yoff = self.oscilo.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', 	separator=';')
        self.oscilo.write('DATa:SOUrce CH1')
        self.oscilo.write('DAT:ENC RPB')
        self.oscilo.write('DAT:WID 1')
        data = self.oscilo.query_binary_values('CURV?', datatype='B', container=np.array)
        tiempo = xze + np.arange(len(data)) * xin
        plt.plot(tiempo, data);
        plt.xlabel('Tiempo [s]');
        plt.ylabel('Voltaje [V]');
        plt.show()
        np.savetxt(nombre+"data.dat",data)
        np.savetxt(nombre+"tiempo.dat",tiempo)


Oscilo=Osciloscopio(device[0])
Channel=1
Volt_set=1
voltaje_trigger=0.5
tiempo=5E-2

Oscilo.pantalla("EntradaArduino-5VPP_PWM100")

sys.exit()

SystemExit: 

C:\Users\publico.LABORATORIOS\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
